In [1]:
import random
import copy
import numpy as np
import pyspiel
from open_spiel.python.algorithms import mcts, random_agent, minimax
from othello_game import OthelloGame, OthelloState
from MCTS import MCTS
from Arena import Arena
from Coach import Coach
from BasicModel import BasicModel
from NNet import NNetWrapper
from QNet import QNetWrapper
from HybridQNet import HybridQNetWrapper
import matplotlib.pyplot as plt

In [2]:
def plot_performance_history(history, name):
    x = [i for i, _ in history]
    random_wins = [results["random"][0] for _, results in history]
    random_draws = [results["random"][2] for _, results in history]
    random_losses = [results["random"][1] for _, results in history]
    mcts_wins = [results["mcts"][0] for _, results in history]
    mcts_draws = [results["mcts"][2] for _, results in history]
    mcts_losses = [results["mcts"][1] for _, results in history]

    fig, axs = plt.subplots(1, 2, figsize=(8, 3))

    axs[0].bar(x, random_wins, color='g', width=2, label='random wins')
    axs[0].bar(x, random_draws, color='b', width=2, bottom=random_wins, label='random draws')
    axs[0].bar(x, random_losses, color='r', width=2, bottom=np.array(random_wins)+np.array(random_draws), label='random losses')

    axs[0].set_xticks(x)

    axs[0].set_xlabel('Iteration')
    axs[0].set_ylabel('Number of games')
    axs[0].set_title(f"{name} vs Random Agent")

    axs[1].bar(x, mcts_wins, color='g', width=2, label='mcts wins')
    axs[1].bar(x, mcts_draws, color='b', width=2, bottom=mcts_wins, label='mcts draws')
    axs[1].bar(x, mcts_losses, color='r', width=2, bottom=np.array(mcts_wins)+np.array(mcts_draws), label='mcts losses')

    axs[1].set_xticks(x)

    axs[1].set_xlabel('Iteration')
    axs[1].set_ylabel('Number of games')
    axs[1].set_title(f"{name} vs MCTS Agent")

In [3]:
game = pyspiel.load_game("othello_nxn", {"n": 6})
hybrid_qnet = HybridQNetWrapper(game, lr=0.0025, use_gpu=False)
print(f"hybrid_qnet params: {hybrid_qnet.total_params}")

Using CPU with lightning.qubit
hybrid_qnet params: 799


In [4]:
hybrid_qnet_coach = Coach(game, hybrid_qnet, numMCTSSims=25, max_history_len=8192, compare_games=50)
hybrid_qnet_coach.learn()

Iter 0
Testing...


Arena.playGames (2): 100%|██████████| 25/25 [02:21<00:00,  5.67s/it]


{'random': (27, 17, 6), 'mcts': (4, 45, 1)}
Iter 1


Self Play: 100%|██████████| 5/5 [00:42<00:00,  8.56s/it]


Epoch 0


...

Epoch 13


Training Net: 100%|██████████| 32/32 [00:01<00:00, 27.60it/s, Loss_v=3.44e-01]


Epoch 14


Training Net:  12%|█▎        | 4/32 [00:00<00:01, 22.40it/s, Loss_v=3.47e-01]


KeyboardInterrupt: 